# Load local dataset

In [1]:
REPO_NAME = "amk-whisper-v5.5"

In [2]:
import torch
torch.cuda.current_device()

0

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset, DatasetDict

In [5]:
dataset = load_dataset("./data/113-sim-dataset/", split="train")

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Using custom data configuration 113-sim-dataset-a5724a57e874f136
Found cached dataset audiofolder (/home/amk/.cache/huggingface/datasets/audiofolder/113-sim-dataset-a5724a57e874f136/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc)


In [6]:
dataset["audio"]

[{'path': '/home/amk/amk/data/113-sim-dataset/audio/113-samtale_part0.wav',
  'array': array([ 0.0007019 ,  0.0007019 ,  0.0007019 , ...,  0.04537964,
          0.        , -0.00387573], dtype=float32),
  'sampling_rate': 8000},
 {'path': '/home/amk/amk/data/113-sim-dataset/audio/113-samtale_part1.wav',
  'array': array([ 0.02828979, -0.02804565, -0.02658081, ...,  0.9076843 ,
          0.70578   ,  0.4340515 ], dtype=float32),
  'sampling_rate': 8000},
 {'path': '/home/amk/amk/data/113-sim-dataset/audio/113-samtale_part10.wav',
  'array': array([ 0.03292847,  0.01608276, -0.01119995, ..., -0.00094604,
         -0.00094604, -0.00094604], dtype=float32),
  'sampling_rate': 8000},
 {'path': '/home/amk/amk/data/113-sim-dataset/audio/113-samtale_part11.wav',
  'array': array([ 0.        ,  0.00119019,  0.00143433, ..., -0.00021362,
         -0.00021362, -0.00045776], dtype=float32),
  'sampling_rate': 8000},
 {'path': '/home/amk/amk/data/113-sim-dataset/audio/113-samtale_part12.wav',
  'ar

In [7]:
dataset = dataset.shuffle(seed=42)

Loading cached shuffled indices for dataset at /home/amk/.cache/huggingface/datasets/audiofolder/113-sim-dataset-a5724a57e874f136/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc/cache-6a6560642171925c.arrow


In [8]:
dataset = dataset.train_test_split(test_size=0.2, shuffle=False)

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'start', 'stop', 'transcription'],
        num_rows: 13
    })
    test: Dataset({
        features: ['audio', 'start', 'stop', 'transcription'],
        num_rows: 4
    })
})

In [10]:
dataset["train"][0]

{'audio': {'path': '/home/amk/amk/data/113-sim-dataset/audio/113-samtale_part7.wav',
  'array': array([-0.00436401, -0.00436401, -0.01119995, ..., -0.00045776,
          0.        ,  0.        ], dtype=float32),
  'sampling_rate': 8000},
 'start': 171,
 'stop': 191,
 'transcription': 'Nå ligger hun på siden. Sjekk om hun fortsatt. Eva! Nå må du sjekke om hun. Hva? Hva sa du? Hva sier du?  Puster normalt. Hva sa du? Sjekk om hun fortsatt puster normalt. Ja, ja. Ja gjør hun det? Ja, hun puster fortsatt normalt. Ja.'}

In [11]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v2")

In [12]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large-v2", language="norwegian", task="transcribe")
tokenizer.push_to_hub(REPO_NAME, private=True, use_auth_token=True)

CommitInfo(commit_url='https://huggingface.co/Jethuestad/amk-whisper-v5.5/commit/8219bc631ef4c335fbb7a29982da87d023a65c2b', commit_message='Upload tokenizer', commit_description='', oid='8219bc631ef4c335fbb7a29982da87d023a65c2b', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2", language="norwegian", task="transcribe")

In [14]:
input_str = dataset["train"][0]["transcription"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 Nå ligger hun på siden. Sjekk om hun fortsatt. Eva! Nå må du sjekke om hun. Hva? Hva sa du? Hva sier du?  Puster normalt. Hva sa du? Sjekk om hun fortsatt puster normalt. Ja, ja. Ja gjør hun det? Ja, hun puster fortsatt normalt. Ja.
Decoded w/ special:    <|startoftranscript|><|no|><|transcribe|><|notimestamps|>Nå ligger hun på siden. Sjekk om hun fortsatt. Eva! Nå må du sjekke om hun. Hva? Hva sa du? Hva sier du?  Puster normalt. Hva sa du? Sjekk om hun fortsatt puster normalt. Ja, ja. Ja gjør hun det? Ja, hun puster fortsatt normalt. Ja.<|endoftext|>
Decoded w/out special: Nå ligger hun på siden. Sjekk om hun fortsatt. Eva! Nå må du sjekke om hun. Hva? Hva sa du? Hva sier du?  Puster normalt. Hva sa du? Sjekk om hun fortsatt puster normalt. Ja, ja. Ja gjør hun det? Ja, hun puster fortsatt normalt. Ja.
Are equal:             True


In [15]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [16]:
dataset["train"][1]

{'audio': {'path': '/home/amk/amk/data/113-sim-dataset/audio/Emil4_VA(i).wav',
  'array': array([-0.00022369, -0.00022327, -0.00073793, ...,  0.00182292,
          0.00216485,  0.        ], dtype=float32),
  'sampling_rate': 16000},
 'start': 0,
 'stop': 29,
 'transcription': 'Sånn. Nå ligger hun ned på siden. Puster hun fortsatt normalt? Hæ? Hva sa du? Puster hun fortsatt normalt? Ja, ja hun puster fortsatt normalt. Nå må du komme snart da, vær så snill. Ja, de er snart til hjelp hos dere. Ja, jeg trenger hjelp. Jeg klarer ikke dette her alene. Jeg vet ikke hva jeg skal gjøre. Du klarer deg veldig fint nå. Kommer det til å gå bra? Det er veldig bra at hun er litt våken nå. Ja. Og at hun puster normalt. Ja, ja. Når er Eva født?'}

In [17]:
def prepare_dataset(batch):
    # load and resample audio data from 8 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch

In [18]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=4)

#1:   0%|          | 0/3 [00:00<?, ?ex/s]

#3:   0%|          | 0/3 [00:00<?, ?ex/s]

#2:   0%|          | 0/3 [00:00<?, ?ex/s]

#0:   0%|          | 0/4 [00:00<?, ?ex/s]

#1:   0%|          | 0/1 [00:00<?, ?ex/s]

#0:   0%|          | 0/1 [00:00<?, ?ex/s]

#2:   0%|          | 0/1 [00:00<?, ?ex/s]

#3:   0%|          | 0/1 [00:00<?, ?ex/s]

In [19]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [20]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [21]:
import evaluate

metric = evaluate.load("wer")


In [22]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [23]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")

In [24]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.dropout = 0.1

In [25]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=REPO_NAME,  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=5e-6,
    warmup_steps=2,
    #max_steps=100,
    num_train_epochs=10,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    #save_steps=20,
    #eval_steps=20,
    logging_steps=1,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [26]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

Cloning https://huggingface.co/Jethuestad/amk-whisper-v5.5 into local empty directory.


In [27]:
trainer.train()

/home/amk/anaconda3/envs/amk/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Epoch,Training Loss,Validation Loss,Wer
1,1.293500,1.084155,38.265306
2,1.293500,1.033903,35.714286
3,1.181100,0.982592,33.163265
4,0.995300,0.910253,28.571429
5,0.799400,0.885159,28.061224
6,0.725700,0.862435,28.571429
7,0.656000,0.842288,28.826531
8,0.597600,0.827379,28.571429
9,0.557200,0.810995,27.806122
10,0.509600,0.804855,28.061224


TrainOutput(global_step=10, training_loss=0.860892379283905, metrics={'train_runtime': 263.9318, 'train_samples_per_second': 0.493, 'train_steps_per_second': 0.038, 'total_flos': 2.76012232704e+17, 'train_loss': 0.860892379283905, 'epoch': 10.0})

In [29]:
trainer.push_to_hub()

Several commits (3) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/5.75G [00:00<?, ?B/s]

To https://huggingface.co/Jethuestad/amk-whisper-v5.5
   8219bc6..4eac23a  main -> main

